# Optimisation Optuna des stages PhysAECe carnet détaille comment explorer et optimiser les stages A, B et la chaîne complète A→B1→B2 avec Optuna. Il fournit des bonnes pratiques pour ajuster les espaces de recherche et automatiser le fine-tuning.

## 1. Préparer l'environnement1. Vérifiez que PhysAE est installé avec le support Optuna :   ```bash   pip install -e .[optuna]   ```2. Activez un serveur d'exécution capable de supporter les essais (CPU ou GPU).3. Créez un répertoire pour stocker les artefacts Optuna (`studies/` par exemple).

In [ ]:
from pathlib import Pathfrom pprint import pprintimport torchfrom physae.config_loader import load_stage_configfrom physae.optimization import optimise_stageROOT = Path.cwd().resolve()ARTIFACTS = ROOT / 'studies'ARTIFACTS.mkdir(exist_ok=True)print(f'Les résultats Optuna seront enregistrés dans {ARTIFACTS.resolve()}')print(f'CUDA disponible ? {torch.cuda.is_available()}')

## 2. Inspecter les espaces de rechercheChaque stage possède une section `optuna` dans son fichier YAML. Affichons les paramètres explorés pour les stages A, B1 et B2.

In [ ]:
for stage in ['A', 'B1', 'B2']:    cfg = load_stage_config(stage)    search_space = cfg.get('optuna', {})    print(f"=== Stage {stage} : {len(search_space)} paramètres ===")    for name, spec in search_space.items():        pprint({name: spec})

## 3. Fonction utilitaire pour lancer une optimisationLe helper ci-dessous centralise les options d'exécution (nombre d'essais, overrides de données ou de stage, accélérateur). Réglez `RUN_OPTUNA = True` pour démarrer une étude complète.

In [ ]:
RUN_OPTUNA = FalseDEFAULT_ACCELERATOR = 'gpu' if torch.cuda.is_available() else 'cpu'def run_optuna_demo(    stage: str,    *,    n_trials: int = 10,    metric: str = 'val_loss',    direction: str = 'minimize',    data_overrides=None,    stage_overrides=None,    accelerator: str | None = None,    study_suffix: str | None = None,):    'Lance une étude Optuna sur un stage PhysAE.'    accelerator = accelerator or DEFAULT_ACCELERATOR    study_name = f"physae-stage-{stage.lower()}"    if study_suffix:        study_name = f"{study_name}-{study_suffix}"    print(f"Stage : {stage}Essais : {n_trials}Accélérateur ciblé : {accelerator}")    print(f"Overrides données : {data_overrides}")    print(f"Overrides stage : {stage_overrides}")    print(f"Nom d'étude Optuna : {study_name}")    if not RUN_OPTUNA:        print('Exécution simulée. Passez RUN_OPTUNA=True pour lancer l'optimisation.')        return None    study = optimise_stage(        stage,        n_trials=n_trials,        metric=metric,        direction=direction,        data_overrides=data_overrides,        stage_overrides=stage_overrides,        output_dir=str(ARTIFACTS),        show_progress_bar=True,    )    print(f"Meilleur score : {study.best_value}")    print('Paramètres optimaux :')    pprint(study.best_params)    return study

## 4. Optimiser le stage ACommencez par réduire la taille des données pour accélérer les essais, puis augmentez progressivement `n_trials`. Ajustez `data_overrides` pour contrôler le bruit, les tailles d'échantillons et les plages de génération.

In [ ]:
run_optuna_demo(    'A',    n_trials=8,    data_overrides={        'n_train': 2048,        'n_val': 512,        'batch_size': 32,    },    stage_overrides={        'epochs': 8,        'enable_progress_bar': True,    },    study_suffix='quickstart',)

## 5. Optimiser le stage B (raffinement)Le stage B1 fige le backbone et se concentre sur le raffineur. Ajustez les paramètres `refine_steps`, `delta_scale` et `refiner_lr`. Les overrides peuvent inclure un checkpoint issu du stage A pour initialiser le modèle.

In [ ]:
run_optuna_demo(    'B1',    n_trials=6,    data_overrides={        'n_train': 1024,        'n_val': 256,    },    stage_overrides={        'epochs': 6,        'ckpt_in': 'artifacts/stage_A.ckpt',        'accelerator': DEFAULT_ACCELERATOR,    },    study_suffix='b1-refiner',)

## 6. Chaîne complète A + B1 + B2 et fine-tuningCe scénario illustre comment enchaîner plusieurs études :1. Optimisez A et récupérez les meilleurs hyperparamètres.2. Lancez B1 avec le checkpoint A optimal.3. Finalisez avec B2 en réactivant le backbone et le module FiLM.Les deux blocs de code suivants montrent comment charger les meilleurs résultats d'une étude précédente et comment configurer une session finale de fine-tuning.

In [ ]:
from pathlib import Pathimport jsondef charger_params_optuna(stage: str, suffix: str):    'Charge les paramètres Optuna sauvegardés pour un stage donné.'    directory = ARTIFACTS / f'stage_{stage.upper()}'    if suffix:        directory = ARTIFACTS / f'stage_{stage.upper()}-{suffix}'    summary_path = directory / 'summary.json'    if not summary_path.exists():        print(f"Aucun résumé trouvé pour {summary_path}")        return None    with open(summary_path, 'r', encoding='utf-8') as fh:        return json.load(fh)best_A = charger_params_optuna('A', 'quickstart')if best_A:    print('Meilleurs hyperparamètres stage A :')    pprint(best_A['best_params'])

In [ ]:
run_optuna_demo(    'B2',    n_trials=4,    data_overrides={        'n_train': 2048,        'n_val': 512,    },    stage_overrides={        'epochs': 8,        'ckpt_in': 'artifacts/stage_B1.ckpt',        'train_base': True,        'train_heads': True,        'train_film': True,        'enable_progress_bar': True,    },    study_suffix='b2-finetune',)

## 7. Bonnes pratiques pour Optuna* **Budget** : démarrez avec peu d'essais (`n_trials=3`) pour valider le pipeline puis augmentez.* **Reproductibilité** : fixez `seed` dans `data_overrides` et utilisez un stockage Optuna (SQLite, PostgreSQL) pour reprendre les études.* **Accélérateur** : forcez `accelerator='cpu'` pour déboguer ; sur GPU, surveillez la mémoire avec `nvidia-smi`.* **Export** : exploitez les fichiers `summary.json`, `best_stage_params.yaml` et `trials.csv` générés automatiquement.* **Coordination des stages** : alignez les plages de recherche entre A et B pour éviter des combinaisons incohérentes (par exemple `width_mult`).